In [1]:
type direction = Left | Right
type state = string

type direction = Left | Right


type state = string


In [2]:
module type TAPE = sig
  type t

  val make : string -> t
  val move : direction -> t -> t
  val read : t -> char
  val write : char -> t -> t
  val print : t -> unit
end

module type TAPE =
  sig
    type t
    val make : string -> t
    val move : direction -> t -> t
    val read : t -> char
    val write : char -> t -> t
    val print : t -> unit
  end


In [3]:
module Tape : TAPE = struct
  type t = {left : char list ; right : char list}
  let string_to_char_list s =
    List.of_seq (String.to_seq s)

  let make s =
    {left  = []; right = string_to_char_list s}

  let move (d : direction) (tape : t) =
    match d with
    | Left -> (match tape.left with
      | [] -> {left = []; right = ' ' :: tape.right}
      | x :: xs -> {left = xs; right = x :: tape.right})
    | Right -> (match tape.right with
      | [] -> {left = ' ' :: tape.left ; right = []}
      | x :: xs -> {left = x :: tape.left ; right = xs})

  let read (tape : t) =
    match tape.right with
    | [] -> ' '
    | x :: xs -> x

  let write (x : char) (tape : t) =
    match tape.right with
    | [] -> {left = tape.left; right = [x]}
    | y :: ys -> {left = tape.left; right = x :: ys }
  
  let print (tape : t) =
    let left_str = String.concat "" (List.rev_map (String.make 1) tape.left) in
    let right_str = String.concat "" (List.map (String.make 1) tape.right) in
    let full_tape = left_str ^ right_str in
    let head_position = String.length left_str in
    let marker = String.make head_position ' ' ^ "^" in
    print_endline (full_tape ^ "\n" ^ marker)

end

module Tape : TAPE


In [4]:
Tape.(
  make "ABCDE"
  |> move Right
  |> move Right
  |> write '!'
  |> move Left
  |> print
)

AB!DE
 ^


- : unit = ()


In [5]:
module type MACHINE = sig
  type t
  val make : state -> state list -> t
  val initial : t -> state
  val add_transition : state -> char -> state -> char -> direction -> t -> t
  val step : t -> state -> Tape.t -> (state * Tape.t) option
end

module type MACHINE =
  sig
    type t
    val make : state -> state list -> t
    val initial : t -> state
    val add_transition :
      state -> char -> state -> char -> direction -> t -> t
    val step : t -> state -> Tape.t -> (state * Tape.t) option
  end


In [6]:
module Machine : MACHINE = struct
  type t = {state_list : state list ; initial : state ; delta : (state * char * direction) option array array; state_index : state -> int }
  module StateMap = Map.Make(String)

  let make_state_to_int_func (states : string list) : (string -> int) =
    let state_map = 
      List.fold_left (fun map (i, state) -> StateMap.add state i map) StateMap.empty (List.mapi (fun i state -> (i, state)) states)
    in
    fun state -> StateMap.find state state_map

  let make initial_state states =
    let all_states = initial_state :: states in
    let num_symbols = 256 in
    let num_states = List.length all_states in
    let transitions = Array.make_matrix num_states num_symbols None in 
    {state_list = all_states; initial = initial_state; delta = transitions; state_index = make_state_to_int_func all_states}

  let initial t =
    t.initial
  
  let add_transition input_state input_char output_state output_char output_direction tm =
    let s_index = tm.state_index input_state in
    let char_index = int_of_char input_char in
    let kopija = Array.copy tm.delta in
    kopija.(s_index).(char_index) <- Some (output_state, output_char, output_direction);
    {state_list = tm.state_list ; initial = tm.initial; delta = kopija ; state_index = tm.state_index} 
  
  let step tm state tape =
    let s_index = tm.state_index state in
    let head = Tape.read tape in
    let char_index = int_of_char head in
    match tm.delta.(s_index).(char_index) with
    | None -> None
    | Some (output_state, output_char, output_direction) -> Some (output_state, Tape.move output_direction (Tape.write output_char tape) )
end

module Machine : MACHINE


In [7]:
let binary_increment =
  Machine.(
    make "right" [ "carry"; "done" ]
    |> add_transition "right" '1' "right" '1' Right
    |> add_transition "right" '0' "right" '0' Right
    |> add_transition "right" ' ' "carry" ' ' Left
    |> add_transition "carry" '1' "carry" '0' Left
    |> add_transition "carry" '0' "done" '1' Left
    |> add_transition "carry" ' ' "done" '1' Left
  )

val binary_increment : Machine.t = <abstr>


In [8]:
let slow_run (tm : Machine.t) str = 
  let initial_tape = Tape.make str in
  Tape.print initial_tape ;
  print_endline (Machine.initial tm);
  let rec aux t s tape =
    match Machine.step t s tape with 
    | None -> print_endline("Machine halted") ;
    | Some (s1, tape1) -> 
      Tape.print tape1 ;
      print_endline(s1) ;
      aux t s1 tape1
    in
  aux tm (Machine.initial tm) initial_tape


val slow_run : Machine.t -> string -> unit = <fun>


In [9]:
let speed_run (tm : Machine.t) str = 
  let initial_tape = Tape.make str in
  let rec aux t s tape =
    match Machine.step t s tape with 
    | None -> Tape.print tape;
    | Some (s1, t1) ->
      aux t s1 t1
    in
  aux tm (Machine.initial tm) initial_tape

val speed_run : Machine.t -> string -> unit = <fun>


In [10]:
let test_run (tm : Machine.t) str n= 
  let initial_tape = Tape.make str in
  Tape.print initial_tape ;
  print_endline (Machine.initial tm);
  let rec aux turing state tape m =
    match m with 
    | 0 -> print_endline "X"
    | m ->  match Machine.step turing state tape with 
    | None -> print_endline ("HALTS") ;
    | Some (s1, t1) -> 
      Tape.print t1 ;
      print_endline (s1);
      aux turing s1 t1 (m - 1)
    in 
    aux tm (Machine.initial tm) initial_tape n

val test_run : Machine.t -> string -> int -> unit = <fun>


In [16]:
let primer_slow_run =
  slow_run binary_increment "1011" 

1011
^
right
1011
 ^
right
1011
  ^
right
1011
   ^
right
1011
    ^
right
1011 
   ^
carry
1010 
  ^
carry
1000 
 ^
carry
1100 
^
done
Machine halted


val primer_slow_run : unit = ()


In [12]:
let primer_speed_run =
  speed_run binary_increment "1011"

1100 
^


val primer_speed_run : unit = ()


In [13]:
let for_state (s : state) (instructions : (state -> Machine.t -> Machine.t) list list) (tm : Machine.t) =
  let all_functions = List.concat instructions in 
  List.fold_left (fun acc f -> f s acc) tm all_functions

let for_character (c : char) (f : char -> state -> Machine.t -> Machine.t) =
  [f c]
  
let for_characters (chars : string) (f : char -> state -> Machine.t -> Machine.t) =
  List.init (String.length chars) (fun i -> f (String.get chars i))
  
let write_and_move (c : char) (d : direction) =
  fun in_char state -> Machine.add_transition state in_char state c d
  
let move (d : direction) =
  fun c state ->
    Machine.add_transition state c state c d
      
let switch_and_move (s : state) (d : direction) =
  fun c in_state -> 
    Machine.add_transition in_state c s c d
  
let write_switch_and_move (c : char) (s : state) (d : direction) =
  fun in_char in_state ->
    Machine.add_transition in_state in_char s c d



val for_state :
  state ->
  (state -> Machine.t -> Machine.t) list list -> Machine.t -> Machine.t =
  <fun>


val for_character :
  char ->
  (char -> state -> Machine.t -> Machine.t) ->
  (state -> Machine.t -> Machine.t) list = <fun>


val for_characters :
  string ->
  (char -> state -> Machine.t -> Machine.t) ->
  (state -> Machine.t -> Machine.t) list = <fun>


val write_and_move :
  char -> direction -> char -> state -> Machine.t -> Machine.t = <fun>


val move : direction -> char -> state -> Machine.t -> Machine.t = <fun>


val switch_and_move :
  state -> direction -> char -> state -> Machine.t -> Machine.t = <fun>


val write_switch_and_move :
  char -> state -> direction -> char -> state -> Machine.t -> Machine.t =
  <fun>


In [14]:
let binary_increment' =
  Machine.make "right" ["carry"; "done"]
  |> for_state "right" [
    for_characters "01" @@ move Right;
    for_character ' ' @@ switch_and_move "carry" Left
  ]
  |> for_state "carry" [
    for_character '1' @@ write_and_move '0' Left;
    for_characters "0 " @@ write_switch_and_move '1' "done" Left
  ] 

val binary_increment' : Machine.t = <abstr>


In [15]:
let primer_speed_run2 =
  speed_run binary_increment' "101011"

101100 
  ^


val primer_speed_run2 : unit = ()
